In [22]:
import pandas as pd
import sqlalchemy
from MyCreds.mycreds import Capstone_AWS_PG
from sqlalchemy import create_engine
import numpy as np

In [44]:
engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)

# Switch depending on which table you want to read.
# sql_query = 'select * from resale_prices_based_on_registration_date;'
sql_query = 'select * from resale_prices_based_on_approval_date;'

with engine.connect() as cnxn:
    approval = pd.read_sql(sql_query, cnxn)


approval

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,n_rooms,storey_range_min,storey_range_max
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN,NaN,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN,NaN,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN,NaN,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN,NaN,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313689,2012-02-01,GEYLANG,4 ROOM,319,UBI AVE 1,10 TO 12,84.0,Simplified,1985,383000.0,NaN,10.0,12.0
1313690,2012-02-01,GEYLANG,4 ROOM,314,UBI AVE 1,01 TO 03,84.0,Simplified,1985,382000.0,NaN,1.0,3.0
1313691,2012-02-01,GEYLANG,4 ROOM,343,UBI AVE 1,01 TO 03,84.0,Simplified,1986,380000.0,NaN,1.0,3.0
1313692,2012-02-01,HOUGANG,3 ROOM,631,HOUGANG AVE 8,07 TO 09,60.0,Improved,1985,300000.0,NaN,7.0,9.0


In [45]:
def clean_approval_df(df):

    df['month'] = pd.to_datetime(df['month'])
    df['n_rooms'] = df['flat_type'].str.extract('(\d)').astype('Int64')
    df['n_rooms'] = df['n_rooms'].astype('Int64')
    # Convert story range to min and max int columns
    df['storey_range_min'] = df['storey_range'].str[:2].astype(int)
    df['storey_range_max'] = df['storey_range'].str[-2:].astype(int)

    df.sort_values('month', ascending=True, inplace=True)
    df.reset_index(inplace=True, drop=True)

    return df

approval = clean_approval_df(approval)
approval

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,n_rooms,storey_range_min,storey_range_max
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,1,10,12
1,1990-01-01,YISHUN,5 ROOM,229,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,98700.0,5,1,3
2,1990-01-01,YISHUN,5 ROOM,231,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,94000.0,5,1,3
3,1990-01-01,YISHUN,5 ROOM,221,YISHUN ST 21,07 TO 09,121.0,IMPROVED,1985,98000.0,5,7,9
4,1990-01-01,YISHUN,5 ROOM,161,YISHUN ST 11,10 TO 12,121.0,IMPROVED,1985,120000.0,5,10,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313689,2012-02-01,SENGKANG,4 ROOM,268A,COMPASSVALE LINK,16 TO 18,90.0,Premium Apartment,2006,525000.0,4,16,18
1313690,2012-02-01,SENGKANG,4 ROOM,268D,COMPASSVALE LINK,13 TO 15,90.0,Premium Apartment,2006,535000.0,4,13,15
1313691,2012-02-01,SENGKANG,4 ROOM,269D,COMPASSVALE LINK,16 TO 18,90.0,Premium Apartment,2006,563000.0,4,16,18
1313692,2012-02-01,SENGKANG,4 ROOM,184C,RIVERVALE CRES,16 TO 18,86.0,Model A2,2003,423000.0,4,16,18


In [42]:
engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)

# Switch depending on which table you want to read.
sql_query = 'select * from resale_prices_based_on_registration_date;'
# sql_query = 'select * from resale_prices_based_on_approval_date;'

with engine.connect() as cnxn:
    registration = pd.read_sql(sql_query, cnxn)


In [52]:
def clean_registration(df):
    df['n_rooms'] = df['flat_type'].str.extract('(\d)').astype('Int64')
    df['n_rooms'] = df['n_rooms'].astype('Int64')
    # Convert story range to min and max int columns
    df['storey_range_min'] = df['storey_range'].str[:2].astype(int)
    df['storey_range_max'] = df['storey_range'].str[-2:].astype(int)

    # Convert remaining lease years to int years and int months remaining columns
    df['remaining_lease_years_reg'] = df['remaining_lease'].str.extract('(?P<remaining_lease_years>\d{2}) year[s]?')
    df[['remaining_lease_months_reg', 'remaining_lease_month']] = df['remaining_lease'].str.extract('\d{2} year[s]? (?P<remaining_lease_months>\d{2}) month[s]?|(?P<remaining_lease_month>\d{2}) month[s]?')
    df.drop('remaining_lease_month', axis=1, inplace=True)
    df[['remaining_lease_years', 'remaining_lease_months']] = df[['remaining_lease_years', 'remaining_lease_months']].astype('Int64').copy()

    df.sort_values('month', ascending=True, inplace=True)
    df.reset_index(inplace=True, drop=True)
    df['remaining_lease_months'].fillna(pd.Series(np.where(df['remaining_lease'].isnull(), np.nan, 0)), inplace=True)

    return df

registration = clean_registration(registration)
registration

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,n_rooms,storey_range_min,storey_range_max,remaining_lease_years,remaining_lease_months,remaining_lease_years_reg,remaining_lease_months_reg
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,None,2,6,10,<NA>,<NA>,NaN,NaN
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,None,2,1,5,<NA>,<NA>,NaN,NaN
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,None,3,6,10,<NA>,<NA>,NaN,NaN
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,None,3,1,5,<NA>,<NA>,NaN,NaN
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,None,3,6,10,<NA>,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210825,2022-03,YISHUN,5 ROOM,504C,YISHUN ST 51,10 TO 12,112.0,Improved,2016,630000.0,93 years 01 month,5,10,12,93,1,93,01
210826,2022-03,YISHUN,5 ROOM,501B,YISHUN ST 51,07 TO 09,112.0,Improved,2018,652000.0,94 years 11 months,5,7,9,94,11,94,11
210827,2022-03,YISHUN,5 ROOM,820,YISHUN ST 81,07 TO 09,122.0,Improved,1988,585000.0,65 years 06 months,5,7,9,65,6,65,06
210828,2022-03,YISHUN,4 ROOM,504C,YISHUN ST 51,10 TO 12,92.0,Model A,2016,490000.0,93 years 01 month,4,10,12,93,1,93,01


In [85]:
rs_df = pd.concat([approval, registration])

rs_df['month'] = pd.to_datetime(rs_df['month'])
rs_df['lease_commence_date'] = pd.to_datetime(rs_df['lease_commence_date'], format='%Y')

# Remaining lease years =
# if month >= 6
# lease_commence_date + 99 - month
# if month < 6
# lease_commence_date + 98 - month
rs_df['remaining_lease_years'] = np.where(rs_df['month'].dt.month >= 6, ((rs_df['lease_commence_date'].dt.year + 98) - rs_df['month'].dt.year), ((rs_df['lease_commence_date'].dt.year + 99) - rs_df['month'].dt.year))

# Convert back to int year.
rs_df['lease_commence_date'] = rs_df['lease_commence_date'].dt.year.astype(int)


# Price per square foot and meter
rs_df['price_per_sq_ft'] = rs_df['resale_price'] / (rs_df['floor_area_sqm'] * 3.28084) #3.28084 ft per meter
rs_df['price_per_sq_m'] = rs_df['resale_price'] / rs_df['floor_area_sqm']

# Price per square foot/meter and remaining lease years
rs_df['price_per_sq_ft_per_lease_yr'] = rs_df['price_per_sq_ft'] / rs_df['remaining_lease_years']
rs_df['price_per_sq_m_per_lease_yr'] = rs_df['price_per_sq_m'] / rs_df['remaining_lease_years']



rs_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,storey_range_max,remaining_lease,remaining_lease_years,remaining_lease_months,remaining_lease_years_reg,remaining_lease_months_reg,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,...,12,NaN,86,<NA>,NaN,NaN,88.490320,290.322581,1.028957,3.375844
1,1990-01-01,YISHUN,5 ROOM,229,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,98700.0,...,3,NaN,94,<NA>,NaN,NaN,248.626108,815.702479,2.644959,8.677686
2,1990-01-01,YISHUN,5 ROOM,231,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,94000.0,...,3,NaN,94,<NA>,NaN,NaN,236.786769,776.859504,2.519008,8.264463
3,1990-01-01,YISHUN,5 ROOM,221,YISHUN ST 21,07 TO 09,121.0,IMPROVED,1985,98000.0,...,9,NaN,94,<NA>,NaN,NaN,246.862802,809.917355,2.626200,8.616142
4,1990-01-01,YISHUN,5 ROOM,161,YISHUN ST 11,10 TO 12,121.0,IMPROVED,1985,120000.0,...,12,NaN,94,<NA>,NaN,NaN,302.280982,991.735537,3.215755,10.550378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210825,2022-03-01,YISHUN,5 ROOM,504C,YISHUN ST 51,10 TO 12,112.0,Improved,2016,630000.0,...,12,93 years 01 month,93,1,93,01,1714.499945,5625.000000,18.435483,60.483871
210826,2022-03-01,YISHUN,5 ROOM,501B,YISHUN ST 51,07 TO 09,112.0,Improved,2018,652000.0,...,9,94 years 11 months,95,11,94,11,1774.371372,5821.428571,18.677593,61.278195
210827,2022-03-01,YISHUN,5 ROOM,820,YISHUN ST 81,07 TO 09,122.0,Improved,1988,585000.0,...,9,65 years 06 months,65,6,65,06,1461.540937,4795.081967,22.485245,73.770492
210828,2022-03-01,YISHUN,4 ROOM,504C,YISHUN ST 51,10 TO 12,92.0,Model A,2016,490000.0,...,12,93 years 01 month,93,1,93,01,1623.391252,5326.086957,17.455820,57.269752


In [86]:
rs_df['address']=rs_df['block']+' '+rs_df['street_name']

##Deal with short forms
rs_df['address']=rs_df['address'].str.replace(' AVE ',' AVENUE ')
rs_df['address']=rs_df['address'].str.replace(' ST ',' STREET ')
rs_df['address']=rs_df['address'].str.replace(' NTH',' NORTH')
rs_df['address']=rs_df['address'].str.replace(' STH',' SOUTH')
rs_df['address']=rs_df['address'].str.replace(' RD',' ROAD')
rs_df['address']=rs_df['address'].str.replace(' UPP ',' UPPER ')
rs_df['address']=rs_df['address'].str.replace(' CTRL',' CENTRAL')
rs_df['address']=rs_df['address'].str.replace(' JLN ',' JALAN ')
rs_df['address']=rs_df['address'].str.replace('BT ','BUKIT ')
rs_df['address']=rs_df['address'].str.replace(' HTS',' HEIGHTS')
rs_df['address']=rs_df['address'].str.replace(' MKT ',' MARKET ')
rs_df['address']=rs_df['address'].str.replace('TG ','TANJONG ')
rs_df['address']=rs_df['address'].str.replace("C'WEALTH",'COMMONWEALTH')
rs_df['address']=rs_df['address'].str.replace("LOR ",'LORONG ')
rs_df['address']=rs_df['address'].str.replace("GDNS",'GARDENS')
rs_df['address']=rs_df['address'].str.replace('ST. ','SAINT ')
rs_df['address']=rs_df['address'].str.replace(' PK',' PARK')
rs_df['address']=rs_df['address'].str.replace(' DR ',' DRIVE ')
rs_df['address']=rs_df['address'].str.replace('KG ','KAMPONG ')
rs_df['address']=rs_df['address'].str.replace(' CL ',' CLOSE ')

/var/folders/l5/fj2kg32d6rz92n5r1t_xxk880000gn/T/ipykernel_3178/3394884286.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  rs_df['address']=rs_df['address'].str.replace('ST. ','SAINT ')


In [66]:
rs_df['address'] = rs_df['address'] + ", " + rs_df['town'] + ", SINGAPORE"
rs_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,remaining_lease,remaining_lease_years,remaining_lease_months,remaining_lease_years_reg,remaining_lease_months_reg,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr,address
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,...,NaN,86,<NA>,NaN,NaN,88.490320,290.322581,1.028957,3.375844,"309 ANG MO KIO AVENUE 1, ANG MO KIO, SINGAPORE"
1,1990-01-01,YISHUN,5 ROOM,229,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,98700.0,...,NaN,94,<NA>,NaN,NaN,248.626108,815.702479,2.644959,8.677686,"229 YISHUN STREET 21, YISHUN, SINGAPORE"
2,1990-01-01,YISHUN,5 ROOM,231,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,94000.0,...,NaN,94,<NA>,NaN,NaN,236.786769,776.859504,2.519008,8.264463,"231 YISHUN STREET 21, YISHUN, SINGAPORE"
3,1990-01-01,YISHUN,5 ROOM,221,YISHUN ST 21,07 TO 09,121.0,IMPROVED,1985,98000.0,...,NaN,94,<NA>,NaN,NaN,246.862802,809.917355,2.626200,8.616142,"221 YISHUN STREET 21, YISHUN, SINGAPORE"
4,1990-01-01,YISHUN,5 ROOM,161,YISHUN ST 11,10 TO 12,121.0,IMPROVED,1985,120000.0,...,NaN,94,<NA>,NaN,NaN,302.280982,991.735537,3.215755,10.550378,"161 YISHUN STREET 11, YISHUN, SINGAPORE"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210825,2022-03-01,YISHUN,5 ROOM,504C,YISHUN ST 51,10 TO 12,112.0,Improved,2016,630000.0,...,93 years 01 month,93,1,93,01,1714.499945,5625.000000,18.435483,60.483871,"504C YISHUN STREET 51, YISHUN, SINGAPORE"
210826,2022-03-01,YISHUN,5 ROOM,501B,YISHUN ST 51,07 TO 09,112.0,Improved,2018,652000.0,...,94 years 11 months,95,11,94,11,1774.371372,5821.428571,18.677593,61.278195,"501B YISHUN STREET 51, YISHUN, SINGAPORE"
210827,2022-03-01,YISHUN,5 ROOM,820,YISHUN ST 81,07 TO 09,122.0,Improved,1988,585000.0,...,65 years 06 months,65,6,65,06,1461.540937,4795.081967,22.485245,73.770492,"820 YISHUN STREET 81, YISHUN, SINGAPORE"
210828,2022-03-01,YISHUN,4 ROOM,504C,YISHUN ST 51,10 TO 12,92.0,Model A,2016,490000.0,...,93 years 01 month,93,1,93,01,1623.391252,5326.086957,17.455820,57.269752,"504C YISHUN STREET 51, YISHUN, SINGAPORE"


In [87]:
rs_df['remaining_lease_years'] = np.where(rs_df['remaining_lease'].isnull(), rs_df['remaining_lease_years'], rs_df['remaining_lease_years_reg'])
rs_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,remaining_lease,remaining_lease_years,remaining_lease_months,remaining_lease_years_reg,remaining_lease_months_reg,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr,address
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,...,NaN,86,<NA>,NaN,NaN,88.490320,290.322581,1.028957,3.375844,309 ANG MO KIO AVENUE 1
1,1990-01-01,YISHUN,5 ROOM,229,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,98700.0,...,NaN,94,<NA>,NaN,NaN,248.626108,815.702479,2.644959,8.677686,229 YISHUN STREET 21
2,1990-01-01,YISHUN,5 ROOM,231,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,94000.0,...,NaN,94,<NA>,NaN,NaN,236.786769,776.859504,2.519008,8.264463,231 YISHUN STREET 21
3,1990-01-01,YISHUN,5 ROOM,221,YISHUN ST 21,07 TO 09,121.0,IMPROVED,1985,98000.0,...,NaN,94,<NA>,NaN,NaN,246.862802,809.917355,2.626200,8.616142,221 YISHUN STREET 21
4,1990-01-01,YISHUN,5 ROOM,161,YISHUN ST 11,10 TO 12,121.0,IMPROVED,1985,120000.0,...,NaN,94,<NA>,NaN,NaN,302.280982,991.735537,3.215755,10.550378,161 YISHUN STREET 11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210825,2022-03-01,YISHUN,5 ROOM,504C,YISHUN ST 51,10 TO 12,112.0,Improved,2016,630000.0,...,93 years 01 month,93,1,93,01,1714.499945,5625.000000,18.435483,60.483871,504C YISHUN STREET 51
210826,2022-03-01,YISHUN,5 ROOM,501B,YISHUN ST 51,07 TO 09,112.0,Improved,2018,652000.0,...,94 years 11 months,94,11,94,11,1774.371372,5821.428571,18.677593,61.278195,501B YISHUN STREET 51
210827,2022-03-01,YISHUN,5 ROOM,820,YISHUN ST 81,07 TO 09,122.0,Improved,1988,585000.0,...,65 years 06 months,65,6,65,06,1461.540937,4795.081967,22.485245,73.770492,820 YISHUN STREET 81
210828,2022-03-01,YISHUN,4 ROOM,504C,YISHUN ST 51,10 TO 12,92.0,Model A,2016,490000.0,...,93 years 01 month,93,1,93,01,1623.391252,5326.086957,17.455820,57.269752,504C YISHUN STREET 51


In [56]:
rs_df.columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price',
       'n_rooms', 'storey_range_min', 'storey_range_max', 'remaining_lease',
       'remaining_lease_years', 'remaining_lease_months',
       'remaining_lease_years_reg', 'remaining_lease_months_reg',
       'price_per_sq_ft', 'price_per_sq_m', 'price_per_sq_ft_per_lease_yr',
       'price_per_sq_m_per_lease_yr', 'address'],
      dtype='object')

In [88]:
rs_df = rs_df[['month', 'block', 'town', 'street_name', 'address', 'storey_range', 'storey_range_min', 'storey_range_max',
      'flat_model', 'flat_type', 'n_rooms', 'floor_area_sqm',
       'lease_commence_date', 'remaining_lease', 'remaining_lease_years', 'remaining_lease_months',
      'resale_price', 'price_per_sq_ft', 'price_per_sq_m', 'price_per_sq_ft_per_lease_yr', 'price_per_sq_m_per_lease_yr']].sort_values('month', ascending=True)

In [69]:
rs_df

,month,block,town,street_name,address,storey_range,storey_range_min,storey_range_max,flat_model,flat_type,...,floor_area_sqm,lease_commence_date,remaining_lease,remaining_lease_years,remaining_lease_months,resale_price,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr
0,1990-01-01,309,ANG MO KIO,ANG MO KIO AVE 1,"309 ANG MO KIO AVENUE 1, ANG MO KIO, SINGAPORE",10 TO 12,10,12,IMPROVED,1 ROOM,...,31.0,1977,NaN,86,<NA>,9000.0,88.490320,290.322581,1.028957,3.375844
2004,1990-01-01,307,CLEMENTI,CLEMENTI AVE 4,"307 CLEMENTI AVENUE 4, CLEMENTI, SINGAPORE",01 TO 03,1,3,NEW GENERATION,3 ROOM,...,67.0,1979,NaN,88,<NA>,47000.0,213.814919,701.492537,2.429715,7.971506
2005,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,"309 CLEMENTI AVENUE 4, CLEMENTI, SINGAPORE",04 TO 06,4,6,NEW GENERATION,3 ROOM,...,82.0,1980,NaN,89,<NA>,57000.0,211.873164,695.121951,2.380597,7.810359
2006,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,"309 CLEMENTI AVENUE 4, CLEMENTI, SINGAPORE",10 TO 12,10,12,NEW GENERATION,3 ROOM,...,67.0,1980,NaN,89,<NA>,34000.0,154.674622,507.462687,1.737917,5.701828
2007,1990-01-01,310,CLEMENTI,CLEMENTI AVE 4,"310 CLEMENTI AVENUE 4, CLEMENTI, SINGAPORE",04 TO 06,4,6,NEW GENERATION,3 ROOM,...,67.0,1980,NaN,89,<NA>,47000.0,213.814919,701.492537,2.402415,7.881939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210529,2022-03-01,208B,CLEMENTI,CLEMENTI AVE 6,"208B CLEMENTI AVENUE 6, CLEMENTI, SINGAPORE",19 TO 21,19,21,Model A,3 ROOM,...,68.0,2017,94 years 08 months,94,8,575000.0,2577.352859,8455.882353,27.418647,89.956195
210528,2022-03-01,320,CLEMENTI,CLEMENTI AVE 4,"320 CLEMENTI AVENUE 4, CLEMENTI, SINGAPORE",04 TO 06,4,6,New Generation,3 ROOM,...,67.0,1979,56 years 02 months,56,2,380888.0,1732.756100,5684.895522,30.942073,101.515991
210527,2022-03-01,334,CLEMENTI,CLEMENTI AVE 2,"334 CLEMENTI AVENUE 2, CLEMENTI, SINGAPORE",07 TO 09,7,9,New Generation,3 ROOM,...,67.0,1978,55 years 05 months,55,5,416000.0,1892.489492,6208.955224,34.408900,112.890095
210538,2022-03-01,40,GEYLANG,CIRCUIT RD,"40 CIRCUIT ROAD, GEYLANG, SINGAPORE",10 TO 12,10,12,Standard,3 ROOM,...,57.0,1971,47 years 11 months,47,11,293000.0,1566.778897,5140.350877,32.641227,107.090643


In [89]:
rs_df.dtypes

month                           datetime64[ns]
block                                   object
town                                    object
street_name                             object
address                                 object
storey_range                            object
storey_range_min                         int64
storey_range_max                         int64
flat_model                              object
flat_type                               object
n_rooms                                  Int64
floor_area_sqm                         float64
lease_commence_date                      int64
remaining_lease                         object
remaining_lease_years                   object
remaining_lease_months                   Int64
resale_price                           float64
price_per_sq_ft                        float64
price_per_sq_m                         float64
price_per_sq_ft_per_lease_yr           float64
price_per_sq_m_per_lease_yr            float64
dtype: object

In [72]:
rs_df['remaining_lease_years'] = rs_df['remaining_lease_years'].astype('Int64')
rs_df.dtypes

TypeError: cannot safely cast non-equivalent object to int64

In [91]:
rs_df[rs_df['remaining_lease_years'].isnull() == True]

,month,block,town,street_name,address,storey_range,storey_range_min,storey_range_max,flat_model,flat_type,...,floor_area_sqm,lease_commence_date,remaining_lease,remaining_lease_years,remaining_lease_months,resale_price,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr
53059,2015-01-01,471,SEMBAWANG,SEMBAWANG DR,471 SEMBAWANG DR,10 TO 12,10,12,Apartment,EXECUTIVE,...,130.0,2000,84,NaN,0,498000.0,1167.618424,3830.769231,13.900219,45.604396
53039,2015-01-01,18C,QUEENSTOWN,HOLLAND DR,18C HOLLAND DR,01 TO 03,1,3,Model A,4 ROOM,...,92.0,2012,96,NaN,0,728000.0,2411.895575,7913.043478,25.123912,82.427536
53038,2015-01-01,181,QUEENSTOWN,STIRLING RD,181 STIRLING ROAD,01 TO 03,1,3,Model A,4 ROOM,...,96.0,2000,84,NaN,0,670000.0,2127.249932,6979.166667,25.324404,83.085317
53037,2015-01-01,589B,SEMBAWANG,MONTREAL DR,589B MONTREAL DR,10 TO 12,10,12,Model A2,4 ROOM,...,85.0,2002,86,NaN,0,352000.0,1262.230548,4141.176471,14.677099,48.153215
53036,2015-01-01,405,SEMBAWANG,ADMIRALTY LINK,405 ADMIRALTY LINK,01 TO 03,1,3,Model A2,4 ROOM,...,86.0,2001,85,NaN,0,343000.0,1215.655775,3988.372093,14.301833,46.922025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88435,2016-12-01,330,CLEMENTI,CLEMENTI AVE 2,330 CLEMENTI AVENUE 2,10 TO 12,10,12,New Generation,3 ROOM,...,82.0,1978,60,NaN,0,453888.0,1687.134853,5535.219512,28.118914,92.253659
88434,2016-12-01,334,CLEMENTI,CLEMENTI AVE 2,334 CLEMENTI AVENUE 2,07 TO 09,7,9,New Generation,3 ROOM,...,67.0,1978,60,NaN,0,385000.0,1751.462631,5746.268657,29.191044,95.771144
88446,2016-12-01,316,CLEMENTI,CLEMENTI AVE 4,316 CLEMENTI AVENUE 4,01 TO 03,1,3,New Generation,4 ROOM,...,91.0,1979,61,NaN,0,553000.0,1852.246095,6076.923077,30.364690,99.621690
88437,2016-12-01,308,CLEMENTI,CLEMENTI AVE 4,308 CLEMENTI AVENUE 4,10 TO 12,10,12,New Generation,3 ROOM,...,67.0,1980,62,NaN,0,355000.0,1614.985023,5298.507463,26.048146,85.459798


In [92]:
rs_df['remaining_lease_years'].fillna(rs_df['remaining_lease'], inplace=True)

In [93]:
rs_df.dtypes

month                           datetime64[ns]
block                                   object
town                                    object
street_name                             object
address                                 object
storey_range                            object
storey_range_min                         int64
storey_range_max                         int64
flat_model                              object
flat_type                               object
n_rooms                                  Int64
floor_area_sqm                         float64
lease_commence_date                      int64
remaining_lease                         object
remaining_lease_years                   object
remaining_lease_months                   Int64
resale_price                           float64
price_per_sq_ft                        float64
price_per_sq_m                         float64
price_per_sq_ft_per_lease_yr           float64
price_per_sq_m_per_lease_yr            float64
dtype: object

In [94]:
rs_df['remaining_lease_years'] = rs_df['remaining_lease_years'].astype(np.int64)

In [95]:
rs_df

,month,block,town,street_name,address,storey_range,storey_range_min,storey_range_max,flat_model,flat_type,...,floor_area_sqm,lease_commence_date,remaining_lease,remaining_lease_years,remaining_lease_months,resale_price,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr
0,1990-01-01,309,ANG MO KIO,ANG MO KIO AVE 1,309 ANG MO KIO AVENUE 1,10 TO 12,10,12,IMPROVED,1 ROOM,...,31.0,1977,NaN,86,<NA>,9000.0,88.490320,290.322581,1.028957,3.375844
2004,1990-01-01,307,CLEMENTI,CLEMENTI AVE 4,307 CLEMENTI AVENUE 4,01 TO 03,1,3,NEW GENERATION,3 ROOM,...,67.0,1979,NaN,88,<NA>,47000.0,213.814919,701.492537,2.429715,7.971506
2005,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,309 CLEMENTI AVENUE 4,04 TO 06,4,6,NEW GENERATION,3 ROOM,...,82.0,1980,NaN,89,<NA>,57000.0,211.873164,695.121951,2.380597,7.810359
2006,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,309 CLEMENTI AVENUE 4,10 TO 12,10,12,NEW GENERATION,3 ROOM,...,67.0,1980,NaN,89,<NA>,34000.0,154.674622,507.462687,1.737917,5.701828
2007,1990-01-01,310,CLEMENTI,CLEMENTI AVE 4,310 CLEMENTI AVENUE 4,04 TO 06,4,6,NEW GENERATION,3 ROOM,...,67.0,1980,NaN,89,<NA>,47000.0,213.814919,701.492537,2.402415,7.881939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210529,2022-03-01,208B,CLEMENTI,CLEMENTI AVE 6,208B CLEMENTI AVENUE 6,19 TO 21,19,21,Model A,3 ROOM,...,68.0,2017,94 years 08 months,94,8,575000.0,2577.352859,8455.882353,27.418647,89.956195
210528,2022-03-01,320,CLEMENTI,CLEMENTI AVE 4,320 CLEMENTI AVENUE 4,04 TO 06,4,6,New Generation,3 ROOM,...,67.0,1979,56 years 02 months,56,2,380888.0,1732.756100,5684.895522,30.942073,101.515991
210527,2022-03-01,334,CLEMENTI,CLEMENTI AVE 2,334 CLEMENTI AVENUE 2,07 TO 09,7,9,New Generation,3 ROOM,...,67.0,1978,55 years 05 months,55,5,416000.0,1892.489492,6208.955224,34.408900,112.890095
210538,2022-03-01,40,GEYLANG,CIRCUIT RD,40 CIRCUIT ROAD,10 TO 12,10,12,Standard,3 ROOM,...,57.0,1971,47 years 11 months,47,11,293000.0,1566.778897,5140.350877,32.641227,107.090643


In [96]:
rs_df.dtypes

month                           datetime64[ns]
block                                   object
town                                    object
street_name                             object
address                                 object
storey_range                            object
storey_range_min                         int64
storey_range_max                         int64
flat_model                              object
flat_type                               object
n_rooms                                  Int64
floor_area_sqm                         float64
lease_commence_date                      int64
remaining_lease                         object
remaining_lease_years                    int64
remaining_lease_months                   Int64
resale_price                           float64
price_per_sq_ft                        float64
price_per_sq_m                         float64
price_per_sq_ft_per_lease_yr           float64
price_per_sq_m_per_lease_yr            float64
dtype: object

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)

sql_alc_cnxn = engine.connect()

rs_df.to_sql('resale_prices', sql_alc_cnxn, if_exists='replace', index=False)

sql_alc_cnxn.close()